In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import numpy as np
import math
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
# constants

NON_WAV_PER_MILE = 1.161
NON_WAV_PER_MIN = 0.529

WAV_PER_MILE = 1.504
WAV_PER_MIN = 0.529

OUT_CITY_NON_WAV_PER_MILE = 1.348
OUT_CITY_WAV_PER_MILE = 1.746
OUT_CITY_PER_MIN = 0.613

JUNO = "HV0002"
UBER = "HV0003"
VIA = "HV0004"
LYFT = "HV0005"

In [3]:
# import the taxi zone data

taxi_zones = pd.read_csv("/Users/aya/Desktop/NYTWA/taxi_zones.csv")[["zone", "LocationID", "borough"]]

In [4]:
# import the trip data
feb_2019 = pd.read_parquet("/Users/aya/Desktop/Aya_NYTWA/Data/fhvhv_tripdata_2019-02.parquet", engine = "fastparquet")[["hvfhs_license_num", "PULocationID", "DOLocationID", "trip_miles", "trip_time", "base_passenger_fare", "driver_pay", "wav_match_flag", "shared_match_flag"]]
april_2022 = pd.read_parquet("/Users/aya/Desktop/Aya_NYTWA/Data/fhvhv_tripdata_2022-04.parquet", engine = "fastparquet")[["hvfhs_license_num", "PULocationID", "DOLocationID", "trip_miles", "trip_time", "base_passenger_fare", "driver_pay", "wav_match_flag", "shared_match_flag"]]

# February 2019

## Cleaning

In [37]:
df = feb_2019

In [38]:
len(df[df["trip_miles"] <= 0])/len(df)

0.0003322072580415536

In [39]:
len(df[df["trip_time"] <= 0])/len(df)

0.0001702952839863601

In [40]:
len(df[df["base_passenger_fare"] <= 0.01])/len(df)

0.004472371834816849

In [41]:
len(df[df["driver_pay"] <= 0.01])/len(df)

0.10823626965129697

In [42]:
len(df[df["trip_time"] > 5*60*60])/len(df)

1.602253909921186e-05

In [44]:
num_before = len(df)
num_before

20159102

In [45]:
# drop rows where the trip miles or time is less than or equal to 0
df = df.drop(df[df["trip_miles"] <= 0].index)
df = df.drop(df[df["trip_time"] <= 0].index)

# drop rows where the trip time is longer than 5 hours
df = df.drop(df[df["trip_time"] > 5*60*60].index)

# drop rows where the base passenger fare or driver pay is less than $0.01
df = df.drop(df[df["base_passenger_fare"] <= 0.01].index)
df = df.drop(df[df["driver_pay"] <= 0.01].index)

In [46]:
num_after = len(df)
num_dropped = num_before - num_after

In [47]:
num_after

17888790

In [48]:
num_dropped

2270312

In [49]:
num_dropped/num_before

0.11261969903222872

## Adding flags for out of city

In [50]:
df['out_of_city'] = np.where(
    df['DOLocationID'].isnull(), np.NaN, np.where(
    df["DOLocationID"].isin(taxi_zones["LocationID"]), False,True)) 

In [51]:
df["out_of_city"] = df["out_of_city"].astype("bool")

In [52]:
table = pa.Table.from_pandas(df)
pq.write_table(table, '/Users/aya/Desktop/Aya_NYTWA/Data/february_2019_cleaned.parquet')

# April 2022

## Cleaning

In [53]:
df = april_2022

In [54]:
len(df[df["trip_miles"] <= 0])/len(df)

0.00018983176568158252

In [55]:
len(df[df["trip_time"] <= 0])/len(df)

5.632990079572181e-08

In [56]:
len(df[df["base_passenger_fare"] <= 0.01])/len(df)

0.0011658599567690544

In [57]:
len(df[df["driver_pay"] <= 0.01])/len(df)

0.0034525722795713813

In [58]:
len(df[df["trip_time"] > 5*60*60])/len(df)

1.4533114405296227e-05

In [59]:
num_before = len(df)

In [60]:
num_before

17752561

In [61]:
# drop rows where the trip miles or time is less than or equal to 0
df = df.drop(df[df["trip_miles"] <= 0].index)
df = df.drop(df[df["trip_time"] <= 0].index)

# drop rows where the trip time is longer than 5 hours
df = df.drop(df[df["trip_time"] > 5*60*60].index)

# drop rows where the base passenger fare or driver pay is less than $0.01
df = df.drop(df[df["base_passenger_fare"] <= 0.01].index)
df = df.drop(df[df["driver_pay"] <= 0.01].index)

In [62]:
num_after = len(df)
num_dropped = num_before - num_after

In [63]:
num_after

17669379

In [64]:
num_dropped

83182

In [65]:
num_dropped/num_before

0.004685633807989731

## Adding flags for out of city

In [66]:
df['out_of_city'] = np.where(
    df['DOLocationID'].isnull(), np.NaN, np.where(
    df["DOLocationID"].isin(taxi_zones["LocationID"]), False,True)) 

In [67]:
df["out_of_city"] = df["out_of_city"].astype("bool")

In [68]:
table = pa.Table.from_pandas(df)
pq.write_table(table, '/Users/aya/Desktop/Aya_NYTWA/Data/april_2022_cleaned.parquet')